El script de extract.py para extraer datos de resultados de los archivos que usa el servicio web de los resultados. 

Ojo: no se pueden correr en jupyter ya que utilizan archivos de entrada que no están presentes.

In [ ]:
from collections import defaultdict

def value(cont):
    kind = cont[:2]
    cont = cont[2:] # skip activity ID
    points = 0.0
    if '_' in cont:
        parts = cont.split('_')
        if len(parts) == 2:
            up = int(parts.pop(0))
            down = int(parts.pop(0))
            value = up / (1.0 * down)
            points += float(value)
        else:
            assert len(parts) == 3
            whole = int(parts.pop(0))         
            up = int(parts.pop(0))
            down = int(parts.pop(0))
            value = whole + up / (1.0 * down)
            points += float(value)
    elif cont[0] == '#':
        for c in cont:
            if c != '#' and c != '-' and c != '0':
                if c.isdigit():
                    points += int(c)
                elif c == '+': 
                    points += 0.5
                elif c == 'x':
                    points += 1.5
    elif cont != '-':
        points += float(cont)
    return (kind, points)

sem = ["ej17", "ad17", "ej18", "ad18"]
act = ["mc", "eo", "pr", "pf"]

print("sem matr", " ".join(act))

for s in sem:
    filename = "exams_md." + s
    with open(filename) as data:
        for line in data:
            line = line.strip()
            if len(line) > 0:
                t = line.split()
                m = t.pop(0)
                totals = defaultdict(float)
                for token in t:
                    (kind, pts) = value(token)
                    totals[kind] += pts
                print(s, m, " ".join([str(totals.get(a, "NP")) for a in act]))

In [ ]:
El resultado de este script se redirige a un archivo resultados.txt con > en bash.
A este archivo usa como entrada al siguiente script de python.
Este combina información de los distintos semestres (utilizando archivos de listados de inscritos)
y además los puntos de tareas y otros datos capturados en las páginas web de resultados.
Algunos listados contienen solamente terminaciones de matrícula mientras otros contienen los siete dígitos completos.
Herramientas tipo hoja de cálculo suelen tragar ceros iniciales, lo que hay que tomar en cuenta.

In [ ]:
def pad(suffix, goal):
    while len(suffix) < goal:
        suffix = '0' + suffix
    return suffix

def match(suffix, candidates):
    k = 7 - len(suffix)
    for c in candidates:
        if c[k:] == suffix:
            return c
    return None

sem = dict()
ss = ['ej17', 'ad17', 'ej18', 'ad18']
sl = {'ej17': 4, 'ad17': 4, 'ej18': 5, 'ad18': 5}
for s in ss:
    sem[s] = dict()
    with open('inscritos.' + s) as data:
        for line in data:
            tokens = line.split()
            m = tokens.pop(0)
            if m not in sem[s]:
                sem[s][m] = dict()

hdr = ['sem', 'matr']

with open('resultados.txt') as data:
    header = data.readline().split()
    header.pop(0) # sem                                                                                                                                                                                                          
    header.pop(0) # matr                                                                                                                                                                                                         
    hdr += header
    for line in data:
        tokens = line.split()
        s = tokens.pop(0).lower()
        m = tokens.pop(0)
        if  m not in sem[s]:
            continue # oyente                                                                                                                                                                                                    
        for p in range(len(header)):
            sem[s][m][header[p]] = tokens.pop(0).strip()

with open('primera.csv') as data:
    header = data.readline().strip().split(',')
    header.pop(0) # sem                                                                                                                                                                                                          
    header.pop(0) # matr                                                                                                                                                                                                         
    hdr += [h + '_1ra' for h in header]
    for line in data:
        tokens = line.split(',')
        s = tokens.pop(0).lower()
        m = match(pad(tokens.pop(0), sl[s]), sem[s].keys())
        if m is not None: # inscrito                                                                                                                                                                                             
            for p in range(len(header)):
                sem[s][m][header[p] + "_1ra"] = tokens.pop(0).strip()

with open('segunda.csv') as data:
    header = data.readline().strip().split(',')
    header.pop(0) # sem                                                                                                                                                                                                          
    header.pop(0) # matr                                                                                                                                                                                                         
    hdr += [h + '_2da' for h in header]
    for line in data:
        tokens = line.split(',')
        s = tokens.pop(0).lower()
        m = match(tokens.pop(0), sem[s].keys())
        if m is not None: # inscrito                                                                                                                                                                                             
            for p in range(len(header)):
                sem[s][m][header[p] + "_2da"] = tokens.pop(0).strip()

print(' '.join(hdr))
for s in sem:
    for m in sem[s]:
        d = sem[s][m]
        print(s, m, ' '.join([str(d.get(h, "NA")) for h in hdr[2:]]))

In [ ]:
La salida de éste se almacena para procesamiento: 
    
python3 combine.py > elisa.dat

In [ ]:
Lo de md5 que se menciona en la práctica se puede también lograr con python:

In [ ]:
import md5
elisa = "elisa.schaeffer@gmail.com"
hash = md5.new()
hash.update(elisa)
hash.hexdigest()

In [ ]:
El resultado es el mismo 7d3927bc722d4bbf96fb449ab4724c11 que se obtuvo en bash.